# 정부혁신1번가 - 정부혁신 홈페이지
* 정부혁신 제안(목록) > 정부혁신국민포럼 

출처 : https://www.innogov.go.kr/ucms/ogp/sug/list.do?menuNo=300011

```
ㆍ내가 낸 정부혁신 제안이 추천 100표 이상 얻으면, 자동으로 2단계 토론으로 진행됩니다.
ㆍ정책제안이 100표 이상 추천을 받지 못한 경우에도 정부혁신국민포럼 운영위원단 논의를 통해 중요하고 시급한 현안으로 결정된 경우 다음 단계인 토론으로 
   진행될 수 있습니다.
ㆍ제안기간은 90일입니다.
ㆍ욕설 및 비속어를 사용한 제안은 관리자에 의해 삭제 될 수 있습니다.
ㆍ유해하며 폭력적이고 선정적인 내용을 담은 제안은 관리자에 의해 삭제 될 수 있습니다.
ㆍ한번 작성된 제안은 삭제가 불가능합니다.
ㆍ허위 사실이나 타인의 명예를 훼손하는 내용이 포함된 제안은 관리자에 의해 '숨김'처리 또는 삭제될 수 있습니다.
```

* https://www.innogov.go.kr/robots.txt

In [1]:
# 필요한 도구를 불러온다.
# 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
import pandas as pd
# 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러올 수 있습니다.
import requests
# request로 가져온 웹사이트의 html 태그를 파싱하는데 사용합니다.
from bs4 import BeautifulSoup as bs
# 랜덤숫자를 생성한다.
import random
import time
# 대량 데이터 처리시 진행 상황을 표시합니다.
from tqdm import tqdm, trange
# 정규표현식
import re

In [2]:
def get_suggestion_list(month, pnum):
    base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/list.do?pnum={pnum}&menuNo=300011&sugMonthTp={month}&orderKey=registDtDesc"
    response = requests.get( base_url )
    # print(f'month : {month}, pnum: {pnum}')
    if response.status_code == 200:
        html = bs(response.text, 'html.parser')
        tags = html.select('#content > div.suggestion_list > ul')[0].find_all('a')
        if not tags :
            return(suggestion_list)
        else :
            for tag in tags:
                suggestion_list.append(tag['href'])
        pnum += 1
        get_suggestion_list(month, pnum)

In [3]:
suggestion_list = []
pnum = 1
for month in trange(7, 13):
    if month < 10:
        month = f'0{month}'
    month = f'2018{month}'
    get_suggestion_list(month, pnum)
    suggestion_list

for month in trange(1, 3):
    if month < 10:
        month = f'0{month}'
    month = f'2019{month}'
    get_suggestion_list(month, pnum)
    suggestion_list    

100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


In [4]:
print(len(suggestion_list))

suggestion_list[:5]

118


['view.do?menuNo=300011&sgId=1&pnum=1',
 'view.do?menuNo=300011&sgId=28&pnum=1',
 'view.do?menuNo=300011&sgId=26&pnum=1',
 'view.do?menuNo=300011&sgId=25&pnum=1',
 'view.do?menuNo=300011&sgId=24&pnum=1']

In [5]:
# id 추출 미리 해보기
u = 'view.do?menuNo=300011&sgId=150&pnum=1'
u.split('=')[2].split('&')[0]

'150'

In [6]:
def get_suggestion_content(url):
    article = []
    base_url = f"https://www.innogov.go.kr/ucms/ogp/sug/{url}"
    response = requests.get( base_url )
    if response.status_code == 200:
        html = bs(response.text, 'html.parser')
        title = html.select('#content > div.vveiw_box1 > dl > dt')[0].get_text(strip=True)
        desc = html.select('#content > div.vveiw_box1 > div.vveiw_name > ul > li > dl > dd')
        category = desc[0].get_text(strip=True)
        content = html.select('#content > div.vveiw_box1 > div.vveiw_cont > div > pre')[0].get_text(strip=True)
        start = desc[1].get_text(strip=True)
        end = desc[2].get_text(strip=True)
        author = desc[3].get_text(strip=True)
        vote = html.select('#counter')[0].get_text(strip=True)
        sgId = url.split('=')[2].split('&')[0]
        
        article.append(sgId)
        article.append(title)
        article.append(category)
        article.append(content)
        article.append(start)
        article.append(end)
        article.append(vote)
        article.append(author)
        
        time.sleep(random.randint(1,2))
        return article

In [7]:
data = []
for i, suggestion in tqdm(enumerate(suggestion_list)):
    s = get_suggestion_content(suggestion)
    data.append(s)

118it [03:35,  1.82s/it]


In [8]:
column_names = ["sgId", "title", "category", "content", "start", "end", "vote", "author"]
column_names

['sgId', 'title', 'category', 'content', 'start', 'end', 'vote', 'author']

In [9]:
data = pd.DataFrame(data, columns=column_names)
data.head()

,sgId,title,category,content,start,end,vote,author
0,1,어린이집 차량내 어린이 질식사고 등에 대한 상세한 대책이 필요하다.,안전,"최근 동두천 어린이집 차량내 어린이 질식사고가 발생하여, 이에 대한 여러가지 대책을...",2018.07.24,2018.08.23,14,헌비
1,28,성희롱 없는 세상,안전,현재 우리 사회에서 익숙해진 대형 범죄가 있습니다.바로 몰래카메라입니다.현재 공공건...,2018.08.31,2018.11.28,3,장대연
2,26,행정안전부소속의 온라인 일자리뱅크운영,상생협력,정부각부처에서는 제각각 일자리창출에 혼신의 힘을 쏟고 있습니다.문제는 제각각하다보니...,2018.08.28,2018.11.25,4,생태조성
3,25,"농어업,농어촌 특별위원회 설치",국민참여,청년실업률이 높아지고 일자리 찾기는 점점 어려워지면서농업과 농촌으로 눈을 돌리는 분...,2018.08.19,2018.11.16,15,생태조성
4,24,혁신적인 귀농창업교육의 패러다임 제시,공동체,"농촌의 생력을 불어넣을 대안으로 귀농귀촌지원정책은 정부,지자체,농협등의 관련기관들이...",2018.08.16,2018.11.13,5,생태조성


In [10]:
data["category"].value_counts()

기타       22
일반행정     21
보건복지     16
안전       15
상생협력     10
인권/평등     9
국민참여      7
반부패청렴     6
환경        5
사회통합      5
공동체       2
Name: category, dtype: int64

In [11]:
data.to_csv("data/suggestion.csv", index=False)

In [13]:
pd.read_csv("data/suggestion.csv").head()

,sgId,title,category,content,start,end,vote,author
0,1,어린이집 차량내 어린이 질식사고 등에 대한 상세한 대책이 필요하다.,안전,"최근 동두천 어린이집 차량내 어린이 질식사고가 발생하여, 이에 대한 여러가지 대책을...",2018.07.24,2018.08.23,14,헌비
1,28,성희롱 없는 세상,안전,현재 우리 사회에서 익숙해진 대형 범죄가 있습니다.바로 몰래카메라입니다.현재 공공건...,2018.08.31,2018.11.28,3,장대연
2,26,행정안전부소속의 온라인 일자리뱅크운영,상생협력,정부각부처에서는 제각각 일자리창출에 혼신의 힘을 쏟고 있습니다.문제는 제각각하다보니...,2018.08.28,2018.11.25,4,생태조성
3,25,"농어업,농어촌 특별위원회 설치",국민참여,청년실업률이 높아지고 일자리 찾기는 점점 어려워지면서농업과 농촌으로 눈을 돌리는 분...,2018.08.19,2018.11.16,15,생태조성
4,24,혁신적인 귀농창업교육의 패러다임 제시,공동체,"농촌의 생력을 불어넣을 대안으로 귀농귀촌지원정책은 정부,지자체,농협등의 관련기관들이...",2018.08.16,2018.11.13,5,생태조성
